# Create reach polygons

The following code takes a river polyline shapefile extracted from the HydroAtlas HydroRIVERS database, and splits it into reaches that are scaled based on a linear relationship between upstream drainage area and reach length, meaning that reaches become progressively longer downstream. This relationship was deduced by examining upstream drainage area estimates at specific points along the Ganges River in India and manually measuring the width of an ideal reach at that point. Finally, buffers are applied to each reach line to create a polygon encompassing the entire reach, which is used in a subsequent script to extract river masks from that area. The script is meant to take a .csv path as an input, and iterate through each row to create reach polygons for the given rivers.

Author: James (Huck) Rees; PhD Student, UCSB Geography

Date: August 12th, 2024

## Import packages

In [1]:
import geopandas as gpd
import pandas as pd
from shapely.ops import unary_union, split
from shapely.geometry import LineString, MultiLineString, Point
import os
import matplotlib.pyplot as plt

## Initialize functions

In [4]:
def read_input_csv(csv_path):
    """
    Reads the input CSV containing the necessary parameters for processing.

    Parameters:
    csv_path (str): Path to the input CSV file.

    Returns:
    DataFrame: A DataFrame containing the parameters.
    """
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"CSV file not found: {csv_path}")
    
    df = pd.read_csv(csv_path)
    
    required_columns = ['river_name', 'buffer_width', 'output_folder_directory', 'output_reaches_path']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"Missing required column in CSV: {col}")
    
    return df

def reimport_river_segments(shapefile_path):
    """
    Reimport river segments from a shapefile.

    Parameters:
    shapefile_path (str): Path to the shapefile.

    Returns:
    GeoDataFrame: A GeoDataFrame containing the river segments.
    """
    if not os.path.exists(shapefile_path):
        raise FileNotFoundError(f"Shapefile not found: {shapefile_path}")
    gdf = gpd.read_file(shapefile_path)
    if gdf.is_empty.any():
        raise ValueError("Input shapefile contains empty geometries.")
    return gdf

def transform_to_crs(gdf, epsg_code=3395):
    """
    Transform the CRS of a GeoDataFrame.

    Parameters:
    gdf (GeoDataFrame): GeoDataFrame to transform.
    epsg_code (int): EPSG code of the target CRS.

    Returns:
    GeoDataFrame: GeoDataFrame with transformed CRS.
    """
    return gdf.to_crs(epsg=epsg_code)

def merge_river_segments(river_segs):
    """
    Merge river segments into a single LineString.

    Parameters:
    river_segs (GeoDataFrame): GeoDataFrame containing river segments.

    Returns:
    GeoDataFrame: A GeoDataFrame containing the merged river segments.
    """
    merged_line = unary_union(river_segs.geometry)
    if isinstance(merged_line, MultiLineString):
        all_coords = []
        for line in merged_line.geoms:
            all_coords.extend(line.coords)
        merged_line = LineString(all_coords)
    river_merged = gpd.GeoDataFrame(geometry=[merged_line], crs=river_segs.crs)
    return river_merged

def calculate_reach_len(upland_skm):
    """
    Calculate reach length from upland_skm.

    Parameters:
    upland_skm (float): Upland area in square kilometers.

    Returns:
    float: Reach length in meters.
    """
    return 0.0668 * upland_skm + 14316

def get_upland_skm_at_distance(river_segs, distance):
    """
    Sample UPLAND_SKM at a specified downstream distance.

    Parameters:
    river_segs (GeoDataFrame): GeoDataFrame containing river segments.
    distance (float): Distance downstream in meters.

    Returns:
    float: UPLAND_SKM value at the specified distance.
    """
    cumulative_length = 0
    for _, segment in river_segs.iterrows():
        segment_length = segment['LENGTH_KM'] * 1000  # Convert km to meters
        if cumulative_length + segment_length >= distance:
            return segment['UPLAND_SKM']
        cumulative_length += segment_length
    return river_segs.iloc[-1]['UPLAND_SKM']  # Return the UPLAND_SKM of the last segment if distance exceeds total length

def create_reach_lengths(river_segs):
    """
    Create reach lengths iteratively.

    Parameters:
    river_segs (GeoDataFrame): GeoDataFrame containing river segments.

    Returns:
    list: List of reach lengths in meters.
    """
    reach_lengths = []
    current_distance = 0
    total_length = river_segs['LENGTH_KM'].sum() * 1000  # Total length in meters

    while current_distance < total_length:
        upland_skm = get_upland_skm_at_distance(river_segs, current_distance)
        reach_length = calculate_reach_len(upland_skm)
        reach_lengths.append(reach_length)
        current_distance += reach_length

    return reach_lengths

def cut(line, distance):
    """
    Cut a line at a specified distance from its starting point.

    Parameters:
    line (LineString): The line to be cut.
    distance (float): The distance from the start of the line where the cut should be made.

    Returns:
    tuple: A tuple containing two LineStrings. The first LineString is the portion of the original
           line from the start to the cut point, and the second LineString is the portion from the
           cut point to the end of the original line.
    """
    # If the distance is 0 or greater than the line length, return the line and an empty LineString
    if distance <= 0.0 or distance >= line.length:
        return line, LineString()  # Return the original line and an empty LineString
    
    # Otherwise, split the line
    for i, seg in enumerate(line.coords):
        pd = line.project(Point(seg))
        if pd == distance:
            return LineString(line.coords[:i+1]), LineString(line.coords[i:])
        if pd > distance:
            cp = line.interpolate(distance)
            return (
                LineString(line.coords[:i] + [(cp.x, cp.y)]),
                LineString([(cp.x, cp.y)] + line.coords[i:])
            )

def split_line_with_points(line, points):
    """
    Split a LineString with a list of points.

    Parameters:
    line (LineString): The LineString to split.
    points (list): List of Points at which to split the line.

    Returns:
    list: List of LineStrings resulting from the split.
    """
    segments = []
    current_line = line
    for p in points:
        d = current_line.project(p)
        seg, current_line = cut(current_line, d)
        segments.append(seg)
    segments.append(current_line)
    return segments

def process_river_reaches(folder_name, output_reaches_partial_path, shapefile_partial_path):
    """
    Main function to process river reaches.

    Parameters:
    shapefile_path (str): Path to the input shapefile.
    output_reaches_path (str): Path to save the output shapefile.
    """
    shapefile_path = f"{shapefile_partial_path}/{folder_name}/{folder_name}.shp"
    river_segs = reimport_river_segments(shapefile_path)
    river_segs = transform_to_crs(river_segs)
    river_merged_gdf = merge_river_segments(river_segs)
    river_merged = river_merged_gdf.geometry[0]  # Convert to shapely LineString

    # Create reach lengths and points
    reach_lengths = create_reach_lengths(river_segs)
    cumulative_distances = [sum(reach_lengths[:i+1]) for i in range(len(reach_lengths))]
    points = [river_merged.interpolate(distance) for distance in cumulative_distances]

    # Split the river_merged line at the points
    split_geometries = split_line_with_points(river_merged, points)

    # Create a GeoDataFrame for the split reaches
    river_reaches = gpd.GeoDataFrame(geometry=split_geometries, crs=river_merged_gdf.crs)

    # Add downstream order, reach length, and distance downstream fields
    river_reaches['ds_order'] = range(1, len(river_reaches) + 1)
    river_reaches['reach_len'] = [segment.length for segment in river_reaches.geometry]
    river_reaches['ds_dist'] = river_reaches['reach_len'].cumsum()
    
    # Delete artifact reaches with length that equals 0
    river_reaches = river_reaches[river_reaches['reach_len'] != 0]
    
    # Delete the most downstream reach if it's too small
    river_reaches = river_reaches.sort_values(by='ds_order', ascending=False)
    record_largest_ds_order = river_reaches.iloc[0]
    record_second_largest_ds_order = river_reaches.iloc[1]

    # Compare their 'reach_len' values
    if record_largest_ds_order['reach_len'] < 0.8 * record_second_largest_ds_order['reach_len']:
        # Delete the record with the largest 'ds_order' value
        river_reaches = river_reaches.drop(record_largest_ds_order.name)

    # Reorder columns to move 'geometry' to the end
    cols = [col for col in river_reaches.columns if col != 'geometry'] + ['geometry']
    river_reaches = river_reaches[cols]
    
    output_file_name = f"{output_reaches_partial_path}/{folder_name}/{folder_name}_reaches.shp"

    # Optionally, save the river_reaches to a shapefile
    river_reaches.to_file(output_file_name)

    return river_reaches

def create_buffered_reaches(folder_name, output_buffers_partial_path, river_reaches, buffer_width):
    """
    Create buffers for each river segment.

    Parameters:
    river_reaches (GeoDataFrame): GeoDataFrame containing river reaches.

    Returns:
    GeoDataFrame: A GeoDataFrame containing buffered river reaches.
    """
    # Calculate buffer width as 1/8th of the reach length
    buffer_widths = river_reaches['reach_len'] * buffer_width

    # Create buffers for each segment
    buffers = river_reaches.geometry.buffer(buffer_widths)

    # Create a GeoDataFrame for the buffers
    river_reach_buffers = river_reaches.copy()
    river_reach_buffers['geometry'] = buffers
    river_reach_buffers['bufwid_m'] = buffer_widths
    river_reach_buffers['bufar_m2'] = river_reach_buffers.geometry.area
    
    # Construct the full path to the output shapefile, including a subfolder named river_name
    output_directory = os.path.join(output_buffers_partial_path, folder_name)
    output_buffers_path = os.path.join(output_directory, folder_name + '.shp')

    # Create the directory if it does not exist
    os.makedirs(output_directory, exist_ok=True)
    
    # Save the river_reach_buffers to a shapefile
    river_reach_buffers.to_file(output_buffers_path)

def main(csv_path):
    """
    Main function to process river data based on input from a CSV file.

    Parameters:
    csv_path (str): The path to the input CSV file.
    """
    # Read the input CSV
    input_df = read_input_csv(csv_path)

    # Process each row in the CSV
    for index, row in input_df.iterrows():
        try:
            folder_name = row['river_name']
            buffer_width = row['buffer_width']
            shapefile_path = row['output_folder_directory']
            output_reaches_path = row['output_folder_directory']
            output_buffers_path = row['output_reaches_path']
            
            # Check for missing values in required columns
            if pd.isna(folder_name) or pd.isna(buffer_width) or pd.isna(shapefile_path) or pd.isna(output_reaches_path) or pd.isna(output_buffers_path):
                print(f"Skipping {folder_name or 'Unknown River'} due to missing data.")
                continue
            
            # Process the river reaches
            river_reaches = process_river_reaches(folder_name, output_reaches_path, shapefile_path)
            
            # Create buffered reaches
            create_buffered_reaches(folder_name, output_buffers_path, river_reaches, buffer_width)
        
        except FileNotFoundError as e:
            print(f"Skipping {folder_name or 'Unknown River'}: {e}")
        except ValueError as e:
            print(f"Skipping {folder_name or 'Unknown River'}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred while processing {folder_name or 'Unknown River'}: {e}")

## Enter input variables and run

The user only needs to enter the path to the .csv file they wish to process from. An formatted example for the input datasheet may be found here: (need to add this in later)

In [6]:
csv_path = r"C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\New_Guinea_river_datasheet.csv"  # Replace with the actual path to the CSV file

main(csv_path)

C:\Users\huckr\miniconda3\Lib\site-packages\shapely\linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
C:\Users\huckr\miniconda3\Lib\site-packages\shapely\linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
C:\Users\huckr\miniconda3\Lib\site-packages\shapely\linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
C:\Users\huckr\miniconda3\Lib\site-packages\shapely\linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
C:\Users\huckr\miniconda3\Lib\site-packages\shapely\linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
C:\Users\huckr\miniconda3\Lib\site-packages\shapely\linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  retu